In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Load dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Set experiment
mlflow.set_experiment("Iris_Classification_Comparison")

# Define models
models = {
    "LogisticRegression": LogisticRegression(max_iter=100),
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "SVC": SVC(probability=True)
}

for name, model in models.items():
    with mlflow.start_run(run_name=name) as run:
        # Train model
        model.fit(X_train, y_train)

        # Predictions
        preds = model.predict(X_test)
        proba = model.predict_proba(X_test)

        # Metrics
        acc = accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds, average='macro')
        loss = log_loss(y_test, proba)

        # Logging
        mlflow.log_param("model_type", name)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score_macro", f1)
        mlflow.log_metric("log_loss", loss)

        mlflow.sklearn.log_model(model, name="model",
                                 input_example=X_test[:5])

        print(f"""{name} → Run ID: {run.info.run_id} | 
                Accuracy: {acc:.4f} | F1 (macro): {f1:.4f} | 
                Log Loss: {loss:.4f}""")

2025/08/13 06:56:31 INFO mlflow.tracking.fluent: Experiment with name 'Iris_Classification_Comparison' does not exist. Creating a new experiment.
2025/08/13 06:56:34 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


LogisticRegression → Run ID: 233591a121ff4a7888f0b525745ca1d5 | 
                Accuracy: 1.0000 | F1 (macro): 1.0000 | 
                Log Loss: 0.1111


2025/08/13 06:56:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


RandomForest → Run ID: 907eb395b3a84a92b0844886b570bc1e | 
                Accuracy: 1.0000 | F1 (macro): 1.0000 | 
                Log Loss: 0.0222


2025/08/13 06:56:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


SVC → Run ID: bb45c60091c84a2ea7160e9d8d1491ed | 
                Accuracy: 1.0000 | F1 (macro): 1.0000 | 
                Log Loss: 0.0771


In [2]:
# (replace with run id of best model from prev. cell output)
best_run_id = "233591a121ff4a7888f0b525745ca1d5"
model_uri = f"runs:/{best_run_id}/model"
mlflow.register_model(model_uri=model_uri, name="IrisBestModel")

print(f"\nBest Model Registered Run ID: {best_run_id}")

Successfully registered model 'IrisBestModel'.
2025/08/13 06:57:11 WARNING mlflow.tracking._model_registry.fluent: Run with id 233591a121ff4a7888f0b525745ca1d5 has no artifacts at artifact path 'model', registering model based on models:/m-49ae1bd815774be488e41fcaf6621149 instead



Best Model Registered Run ID: 233591a121ff4a7888f0b525745ca1d5


Created version '1' of model 'IrisBestModel'.


In [3]:
import mlflow
from sklearn.datasets import load_iris

# Load class names
iris = load_iris()
class_names = iris.target_names

# Load model from registry
model = mlflow.sklearn.load_model("models:/IrisBestModel/latest")
# or, mlflow.sklearn.load_model("models:/SecondBestIris/latest")

# Predict
sample = [[5.3, 3.1, 1.3, 0.2]]
pred = model.predict(sample)[0]  # Numeric class label (e.g., 0)
class_label = class_names[pred]  # Convert to class name

print("Prediction:", class_label)

Prediction: setosa


In [4]:
# save model artifact on system
from mlflow.artifacts import download_artifacts

local_path = download_artifacts(
                    artifact_uri="models:/IrisBestModel/1",
                    dst_path="model_dir")
                    
print(f"Model downloaded to: {local_path}")

/Users/monusingh/work-share/code-blogs-articles/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Model downloaded to: /Users/monusingh/work-share/code-blogs-articles/02-machine-learning/03B-mlops-mlflow/model_dir/
